In [1]:
import tensorflow.keras as keras
import numpy as np
from sklearn.utils import resample
import pandas as pd
import pickle
import os
import csv
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.linear_model import LogisticRegression
from statistics import mean
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD
from keras.regularizers import l2
from sklearn.metrics import accuracy_score
MODEL_PATH = './model/'
DATA_PATH = './data/'

2022-09-17 10:58:53.272982: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-17 10:58:53.273512: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [83]:
# original dataset with specific index
# data = pd.read_csv('data/adult.data', na_values=["?"])
# target_dataset = pd.DataFrame(data.iloc[1:10001,].values)
# shadow_dataset = pd.DataFrame(data.iloc[15001:30001,].values)
# attack_test_nonmembers = pd.DataFrame(data.iloc[11001:14001,].values)
# attack_test_members = pd.DataFrame(data.iloc[2001:5001,].values)

In [84]:
def read_data(data_name):
    with np.load(DATA_PATH + data_name) as f:
        train_x, train_y, test_x, test_y = [f['arr_%d' % i] for i in range(len(f.files))]
    return train_x, train_y, test_x, test_y

In [85]:
def old_transform_data(dataset):
    x = dataset.iloc[:,0:dataset.shape[1]-1] # seperate the feature column from class label
    y = dataset.iloc[:,-1] # label column
    

    dim=x.shape[1] # number of feature columns

    #num of classes
    num_classes=2

    for j in range(0,dim):
        if x.iloc[:,j].dtypes == object:   # transform categorical variables
            x.iloc[:,j] = x.iloc[:,j].astype('category') # change datatype to categorical
            x.iloc[:,j] = x.iloc[:,j].cat.codes # change from category name to category number
        else:  #transform numrical variables to standard scaler form 
            sc = StandardScaler()  
            val=np.array(x.iloc[:,j]).reshape(-1,1)
            std_data = sc.fit_transform(val)
            std_data = pd.DataFrame(std_data)
            x.iloc[:,j]=std_data
            print(x.iloc[:,j])

    y = y.astype('category') # change label to categorical
    y = y.cat.codes # change from category name to number

    x=np.array(x)
    y=np.array(y)
    
    
    y=to_categorical(y)
    
    return x, y, dim

In [86]:
def transform_data(dataset, is_synthetic):
    
    if(is_synthetic == False):
        for col in [1,3,5,6,7,8,9,13,14]:
            le = LabelEncoder()
            dataset[col] = le.fit_transform(dataset[col].astype('str'))

        # normalize the values
        x_range = [i for i in range(14)]
        dataset[x_range] = dataset[x_range]/dataset[x_range].max()

        x = dataset[x_range].values
        y = dataset[14].values
    else:
        for col in [1,2,3,4,5,6,7,11,12]:
            le = LabelEncoder()
            dataset[col] = le.fit_transform(dataset[col].astype('str'))

        # normalize the values
        x_range = [i for i in range(12)]
        dataset[x_range] = dataset[x_range]/dataset[x_range].max()

        x = dataset[x_range].values
        y = dataset[12].values
        
    
    dim = x.shape[1]
    
    x=np.array(x)
    y=np.array(y)
    
    
    y=to_categorical(y)
    
    return x, y, dim

In [87]:
def load_target_data(dataset, trian_size, test_ratio, is_synthetic):
    x, y, dim = transform_data(dataset, is_synthetic)
    
    #trainX,testX, trainY, testY = train_test_split(x, y, test_size=test_ratio, random_state=0, stratify=y)
    trainX = x[0:trian_size,]
    testX = x[7000:,]
    trainY = y[0:trian_size,]
    testY = y[7000:,]
    return (trainX, trainY), (testX, testY), dim

In [88]:
def build_simple_mlp(n_class,pix,d):

    model = Sequential()
    model.add(Dense(256, input_dim=pix))
    model.add(Activation("relu"))
    #model.add(Dropout(0.01))
    
    model.add(Dense(256, kernel_regularizer=l2(0.01)))
    model.add(Activation("relu"))
    #model.add(Dropout(0.01))
    
    
    #model.add(Dense(248))
    #model.add(Activation("relu"))
    #model.add(Dropout(0.01))

    model.add(Dense(64))
    model.add(Activation("relu"))
    model.add(Dropout(0.01))
    
    model.add(Dense(n_class, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    act_layer=3
    
    return model, act_layer

In [101]:
def build_dnn(n_class,dim):
    model = Sequential()
    
    model.add(Dense(600, input_dim=dim))
    model.add(Activation("tanh"))
    #model.add(Dropout(0.01))
    
    #model.add(Dense(1024, kernel_regularizer=l2(0.00003)))
    #model.add(Activation("tanh"))
    #model.add(Dropout(0.01))
    
    model.add(Dense(512, kernel_regularizer=l2(0.00003)))
    model.add(Activation("tanh"))
    #model.add(Dropout(0.01))

    model.add(Dense(256, kernel_regularizer=l2(0.00003)))
    model.add(Activation("tanh"))
    #model.add(Dropout(0.01))
    
    model.add(Dense(128, kernel_regularizer=l2(0.00003)))
    model.add(Activation("tanh"))
    #model.add(Dropout(0.01))
    
    model.add(Dense(n_class, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    
    #opt = SGD(lr=0.01, momentum=0.9)
    #model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

    act_layer=6
    
    return model, act_layer

In [90]:
def load_shadow_data(dataset, n_shadow, shadow_size, test_ratio, is_synthetic):
    x, y, _ = transform_data(dataset, is_synthetic)
    
    shadow_indices = np.arange(len(dataset))
    
   
    for i in range(n_shadow):
        shadow_i_indices = np.random.choice(shadow_indices, shadow_size, replace=False)
        shadow_i_x, shadow_i_y = x[shadow_i_indices], y[shadow_i_indices]
        trainX,testX, trainY, testY = train_test_split(shadow_i_x, shadow_i_y, test_size=test_ratio)
        print('shadow_i_trainX = ', len(trainX), 'shadow_i_trainY = ', len(trainY), 'shadow_i_testX = ', len(testX), 'shadow_i_testY = ', len(testY))
        
        np.savez(DATA_PATH + 'shadow_adult{}_data.npz'.format(i), trainX, trainY, testX, testY)

In [104]:
def train_shadow_models(n_shadow, n_class, dim, channel):
    full_sm_train_pred=[]
    full_sm_train_class=[]
    
    full_sm_test_pred=[]
    full_sm_test_class=[]
    
    full_clz_train=[]
    full_clz_test=[]
    
    members=[]
    nonmembers=[]
    
    train_accuracy=[]
    test_accuracy=[]

    for j in range(n_shadow):
        
        print("Shadow Model ", j)
        
        print('Training shadow model {}'.format(j))
        data = read_data('shadow_adult{}_data.npz'.format(j))
        x_shadow_train, y_shadow_train, x_shadow_test, y_shadow_test = data
        #print('x_shadow trian\n', x_shadow_train,'\n y_shadow trian\n', y_shadow_train, '\n x_shadow test\n', x_shadow_test, '\n y_shadow test\n', y_shadow_test)

        #model, act_layer = build_simple_mlp(n_class,dim, channel)
        model, act_layer = build_dnn(n_class,dim)
            
            
        # fit model
        history = model.fit(x_shadow_train, y_shadow_train, epochs=EPS, batch_size=32, validation_data=(x_shadow_test, y_shadow_test), verbose=0)
    
        # evaluate model
        _, train_acc = model.evaluate(x_shadow_train, y_shadow_train, verbose=0)
        _, test_acc = model.evaluate(x_shadow_test, y_shadow_test, verbose=0)
        print("Shadow Train acc : ", (train_acc * 100.0),"Shadow Test acc : ", (test_acc * 100.0))
        train_accuracy.append((train_acc * 100.0))
        test_accuracy.append((test_acc * 100.0))

    
        #train SM
        sm_train_pred=model.predict(x_shadow_train, batch_size=32)
        sm_train_class=np.argmax(y_shadow_train,axis=1)
    
    
        #test SM
        sm_test_pred=model.predict(x_shadow_test, batch_size=32)
        sm_test_class=np.argmax(y_shadow_test,axis=1)
        
     
        full_sm_train_pred.append(sm_train_pred)        
        full_sm_train_class.append(sm_train_class)
        members.append(np.ones(len(sm_train_pred)))
        
        full_sm_test_pred.append(sm_test_pred)        
        full_sm_test_class.append(sm_test_class) 
        nonmembers.append(np.zeros(len(sm_test_pred)))


    full_sm_train_pred = np.vstack(full_sm_train_pred)
    full_sm_train_class = [item for sublist in full_sm_train_class for item in sublist]
    members = [item for sublist in members for item in sublist]
    
    full_sm_test_pred = np.vstack(full_sm_test_pred)
    full_sm_test_class = [item for sublist in full_sm_test_class for item in sublist]
    nonmembers = [item for sublist in nonmembers for item in sublist]
    
    shadow_train_performance=(full_sm_train_pred, np.array(full_sm_train_class))
    shadow_test_performance=(full_sm_test_pred, np.array(full_sm_test_class))


    ###atack data preparation
    attack_x = (full_sm_train_pred,full_sm_test_pred)
    #attack_x = np.vstack(attack_x)
    
    attack_y = (np.array(members).astype('int32'),np.array(nonmembers).astype('int32'))
    #attack_y = np.concatenate(attack_y)
    #attack_y = attack_y.astype('int32')
    
    
    classes = (np.array(full_sm_train_class),np.array(full_sm_test_class))
    #classes = np.array([item for sublist in classes for item in sublist])


    attack_dataset = (attack_x,attack_y,classes)
    shadow_accuracy = (train_accuracy, test_accuracy)

            
    return  shadow_train_performance, shadow_test_performance, attack_dataset, x_shadow_train, y_shadow_train, x_shadow_test, y_shadow_test, model, shadow_accuracy

In [92]:
def define_attack_model(n_class):
    model = Sequential()
    
    model.add(Dense(1))
    model.add(Activation("relu"))
    
    model.add(Dense(1))
    model.add(Activation("relu"))

    model.add(Dense(n_class, activation='softmax'))

    # compile model
    opt = SGD(learning_rate=0.0001, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [93]:
def attack_mlp(pix,d):

    model = Sequential()
    model.add(Dense(64, input_dim=pix))
    model.add(Activation("relu"))
    #model.add(Dropout(0.1))

#     model.add(Dense(32))
#     model.add(Activation("tanh"))
#     model.add(Dropout(0.01))
    
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    act_layer=1
    
    return model, act_layer

In [94]:
def prep_attack_train_data(n_attack_data):

    attack_mem = pd.DataFrame(n_attack_data[0][0])
    attack_nmem = pd.DataFrame(n_attack_data[0][1])
    
    attack_mem_status = pd.DataFrame(n_attack_data[1][0])
    attack_mem_status.columns = ["membership"]
    
    attack_nmem_status = pd.DataFrame(n_attack_data[1][1])
    attack_nmem_status.columns = ["membership"]
    
    real_class_mem = pd.DataFrame(n_attack_data[2][0])
    real_class_mem.columns = ["y"]
    
    real_class_nmem = pd.DataFrame(n_attack_data[2][1])
    real_class_nmem.columns = ["y"]

    memdf = pd.concat([attack_mem,attack_nmem],axis=0)
    memdf = memdf.reset_index(drop=True)

    memstatus =  pd.concat([attack_mem_status,attack_nmem_status],axis=0)
    memstatus = memstatus.reset_index(drop=True)

    realclass = pd.concat([real_class_mem,real_class_nmem],axis=0)
    realclass = realclass.reset_index(drop=True)

    attack_df = pd.concat([memdf,realclass,memstatus],axis=1)
    
    return attack_df

In [95]:
def prep_validation_data(attack_test_data):

    attack_mem = pd.DataFrame(attack_test_data[0][0])
    attack_nmem = pd.DataFrame(attack_test_data[0][1])
    
    attack_mem_status = pd.DataFrame(attack_test_data[1][0])
    attack_mem_status.columns = ["membership"]
    
    attack_nmem_status = pd.DataFrame(attack_test_data[1][1])
    attack_nmem_status.columns = ["membership"]
    
    real_class_mem = pd.DataFrame(attack_test_data[2][0])
    real_class_mem.columns = ["y"]
    
    real_class_nmem = pd.DataFrame(attack_test_data[2][1])
    real_class_nmem.columns = ["y"]
    
    mem_df = pd.concat([attack_mem,real_class_mem],axis=1)
    nmem_df = pd.concat([attack_nmem,real_class_nmem],axis=1)

#     memdf = pd.concat([attack_mem,attack_nmem],axis=0)
#     memdf = memdf.reset_index(drop=True)

#     memstatus =  pd.concat([attack_mem_status,attack_nmem_status],axis=0)
#     memstatus = memstatus.reset_index(drop=True)

#     realclass = pd.concat([real_class_mem,real_class_nmem],axis=0)
#     realclass = realclass.reset_index(drop=True)

#     attack_df = pd.concat([memdf,realclass,memstatus],axis=1)
    
    return mem_df, nmem_df

In [96]:
def load_attack_test_data(members, nonmembers, is_synthetic):
    memberX, memberY, _ = transform_data(members, is_synthetic)
    
    nonmemberX, nonmemberY, _ = transform_data(nonmembers, is_synthetic)
    
    return memberX, memberY, nonmemberX, nonmemberY

In [97]:
def prety_print_result(mem, pred):
    tn, fp, fn, tp = confusion_matrix(mem, pred).ravel()
    print('TP: %d     FP: %d     FN: %d     TN: %d' % (tp, fp, fn, tn))
    if tp == fp == 0:
        print('PPV: 0\nAdvantage: 0')
    else:
        print('PPV: %.4f\nAdvantage: %.4f' % (tp / (tp + fp), tp / (tp + fn) - fp / (tn + fp)))

    return tp, fp, fn, tn, (tp / (tp + fp)), (tp / (tp + fn) - fp / (tn + fp)), ((tp+tn)/(tp+tn+fp+fn)),  (tp / (tp + fn))

In [98]:
def train_attack_model(attack_data, check_membership, n_hidden=50, learning_rate=0.01, batch_size=200, epochs=50, model='nn', l2_ratio=1e-7):

    x, y,  classes = attack_data

    train_x = x[0]
    train_y = y[0]
    test_x = x[1]
    test_y = y[1]
    train_classes = classes[0]
    test_classes = classes[1]
    
    
    checkmem_prediction_vals, checkmem_membership_status, checkmem_class_status = check_membership
    
    checkmem_prediction_vals=np.vstack(checkmem_prediction_vals)
    checkmem_membership_status=np.array([item for sublist in checkmem_membership_status for item in sublist])
    checkmem_class_status=np.array([item for sublist in checkmem_class_status for item in sublist])
    
    train_indices = np.arange(len(train_x))
    test_indices = np.arange(len(test_x))
    unique_classes = np.unique(train_classes)


    predicted_membership, target_membership = [], []
    for c in unique_classes:
        print("Class : ", c)
        c_train_indices = train_indices[train_classes == c]
        c_train_x, c_train_y = train_x[c_train_indices], train_y[c_train_indices]
        c_test_indices = test_indices[test_classes == c]
        c_test_x, c_test_y = test_x[c_test_indices], test_y[c_test_indices]
        c_dataset = (c_train_x, c_train_y, c_test_x, c_test_y)        
        
        full_cx_data=(c_train_x,c_test_x)
        full_cx_data = np.vstack(full_cx_data)

        full_cy_data=(c_train_y,c_test_y)
        full_cy_data = np.array([item for sublist in full_cy_data for item in sublist])
        
        d=1
        pix = full_cx_data.shape[1]
        classifier, _ = attack_mlp(pix,d)
        history = classifier.fit(full_cx_data, full_cy_data, epochs=EPS, batch_size=32, verbose=0)

        #get predictions on real train and test data
        c_indices = np.where(checkmem_class_status==c)
        pred_y = classifier.predict(checkmem_prediction_vals[c_indices])
        print(pred_y)
        c_pred_y = np.argmax(pred_y, axis=1)
        c_target_y = checkmem_membership_status[c_indices]
        
       
        target_membership.append(c_target_y)
        predicted_membership.append(c_pred_y)

    target_membership=np.array([item for sublist in target_membership for item in sublist])
    predicted_membership=np.array([item for sublist in predicted_membership for item in sublist])


    tp, fp, fn, tn, precision, advj, acc, recall = prety_print_result (target_membership,predicted_membership)   
    return tp, fp, fn, tn, precision, advj, acc, recall

In [99]:
def shokri_attack(attack_df, mem_validation, nmem_validation):
    
    predicted_membership, predicted_nmembership, true_membership, TP_idx, TN_idx  = [], [], [], [], []

    class_val = np.unique(attack_df['y'])
    ncval=attack_df.shape[1]-1
    
    for c_val in class_val:

        print(c_val)
        
        filter_rec_all = attack_df[(attack_df['y'] == c_val)]
        filter_rec_idx = np.array(filter_rec_all.index)
        
        attack_feat = filter_rec_all.iloc[:, 0:ncval]
        attack_class = filter_rec_all['membership']
             
        d=1
        pix = attack_feat.shape[1]
        
        attack_model, _ = attack_mlp(pix,d)
        
       
        history = attack_model.fit(attack_feat, attack_class, epochs=EPS, batch_size=32, verbose=0)
        
        mcval=mem_validation.shape[1]-1
        
        
        check_mem_feat = mem_validation[mem_validation['y']==c_val]
        check_nmem_feat = nmem_validation[nmem_validation['y']==c_val]
        
        if (len(check_mem_feat)!=0) and (len(check_nmem_feat)!=0):
        
            check_mem_feat_idx =  np.array(check_mem_feat.index)


            check_nmem_feat_idx =  np.array(check_nmem_feat.index)

            #print(check_nmem_feat_idx)
            #print(np.argmax(mpred,axis=1)==0)


            mpred = attack_model.predict(np.array(check_mem_feat))    
            predicted_membership.append(np.argmax(mpred,axis=1) )

            nmpred = attack_model.predict(np.array(check_nmem_feat))    
            predicted_nmembership.append(np.argmax(nmpred,axis=1) )        



            TP_idx.append(check_mem_feat_idx[np.where(np.argmax(mpred,axis=1)==1)[0]])

            TN_idx.append(check_nmem_feat_idx[np.where(np.argmax(nmpred,axis=1)==0)[0]])

    pred_members = np.array([item for sublist in predicted_membership for item in sublist])
    pred_nonmembers = np.array([item for sublist in predicted_nmembership for item in sublist])
    
    TP_idx_list = np.array([item for sublist in TP_idx for item in sublist])
    TN_idx_list = np.array([item for sublist in TN_idx for item in sublist])
    
    members=np.array(list(pred_members))
    nonmembers=np.array(list(pred_nonmembers))
    
    pred_membership = np.concatenate([members,nonmembers])
    ori_membership = np.concatenate([np.ones(len(members)), np.zeros(len(nonmembers))])
    
    return pred_membership, ori_membership, TP_idx_list, TN_idx_list

In [105]:
def train_target_model(target_dataset, per_class_sample, epoch, act_layer, n_class, is_synthetic, train_size, channel=0, verbose=0, test_ratio=0.3):
    
    (target_trainX, target_trainY), (target_testX, target_testY), dim = load_target_data(target_dataset, train_size, test_ratio, is_synthetic)
    #target_model,_ = build_simple_mlp(n_class,dim, channel)
    target_model,_ = build_dnn(n_class,dim)
    #get_trained_keras_models(model, (target_trainX, target_trainY), (target_testX, target_testY), num_models=1)
    history = target_model.fit(target_trainX, target_trainY, epochs=EPS, batch_size=32, verbose=VERBOSE)
    score = target_model.evaluate(target_testX, target_testY, verbose=VERBOSE)
    _, train_acc = target_model.evaluate(target_trainX, target_trainY, verbose=VERBOSE)
    _, test_acc = target_model.evaluate(target_testX, target_testY, verbose=VERBOSE)
    print('\n', "Target Train acc : ", (train_acc * 100.0),"Target Test acc : ", (test_acc * 100.0))
    #print('\n', 'Model test accuracy:', score[1])
    return target_model, dim

In [103]:
def prepare_attack_test_data(attack_test_members, attack_test_nonmembers, target_model, is_synthetic):
    members = []
    nonmembers = []

    memberX, memberY, nonmemberX, nonmemberY = load_attack_test_data(attack_test_members, attack_test_nonmembers, is_synthetic)

    # member
    target_model_member_pred = target_model.predict(memberX, batch_size=32)
    target_model_member_class = np.argmax(memberY, axis=1)
    target_model_member_pred = np.vstack(target_model_member_pred)
    #target_model_member_class = [item for sublist in target_model_member_class for item in sublist]
    members.append(np.ones(len(target_model_member_pred)))
    members = [item for sublist in members for item in sublist]


    # nonmember
    target_model_nonmember_pred = target_model.predict(nonmemberX, batch_size=32)
    target_model_nonmember_class = np.argmax(nonmemberY, axis=1)
    target_model_nonmember_pred = np.vstack(target_model_nonmember_pred)
    #target_model_nonmember_class = [item for sublist in target_model_nonmember_class for item in sublist]
    nonmembers.append(np.zeros(len(target_model_nonmember_pred)))
    nonmembers = [item for sublist in nonmembers for item in sublist]

    full_attack_test_pred_val = (target_model_member_pred, target_model_nonmember_pred)
    full_attack_test_mem_status = (np.array(members).astype('int32'),np.array(nonmembers).astype('int32'))
    full_attack_test_class_status = (np.array(target_model_member_class),np.array(target_model_nonmember_class))

    print('\n pred', full_attack_test_pred_val)
    print('\n class', full_attack_test_class_status)
    print('\n mem status', full_attack_test_mem_status)

    attack_test_data = (full_attack_test_pred_val, full_attack_test_mem_status,full_attack_test_class_status)
    
    return attack_test_data

In [20]:
# --------------------------------------------Original Data--------------------------------------------------------------#

In [21]:
# original dataset with random index
train_size = 500
attack_test_size = 250
data = pd.read_csv('data/adult.data', na_values=["?"], header=None)
data.dropna(inplace=True)
target_dataset = data.sample(n = 10000, replace = False)
df_rest = data.loc[~data.index.isin(target_dataset.index)]
shadow_dataset = df_rest.sample(n = 12000, replace = False)
df_rest = df_rest.loc[~df_rest.index.isin(shadow_dataset.index)]
attack_test_nonmembers = df_rest.sample(n = attack_test_size, replace = False)
attack_test_members =  target_dataset.iloc[:train_size,:].sample(n = attack_test_size, replace = False)

In [22]:
# trian target model
per_class_sample=5000
channel=0   
EPS=200
act_layer=3
n_class = 2
is_synthetic = False
VERBOSE = 0
test_ratio = 0.3

target_model, dim = train_target_model(target_dataset, per_class_sample, EPS, act_layer, n_class, is_synthetic, train_size)

2022-09-09 16:54:15.839590: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-09-09 16:54:15.839645: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (msnkhan-desktop): /proc/driver/nvidia/version does not exist
2022-09-09 16:54:15.840312: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.



 Target Train acc :  94.40000057220459 Target Test acc :  80.0000011920929


In [404]:
#train shadow model
n_shadow_models = 20
shadow_data_size = 750

load_shadow_data(shadow_dataset, n_shadow_models, shadow_data_size, test_ratio, is_synthetic)
n_shadow_train_performance, n_shadow_test_performance, n_attack_data, x_shadow_train, y_shadow_train, x_shadow_test, y_shadow_test, shadow_model_init, shadow_accuracy = train_shadow_models(n_shadow_models, n_class, dim, channel)

shadow_i_trainX =  525 shadow_i_trainY =  525 shadow_i_testX =  225 shadow_i_testY =  225
shadow_i_trainX =  525 shadow_i_trainY =  525 shadow_i_testX =  225 shadow_i_testY =  225
shadow_i_trainX =  525 shadow_i_trainY =  525 shadow_i_testX =  225 shadow_i_testY =  225
shadow_i_trainX =  525 shadow_i_trainY =  525 shadow_i_testX =  225 shadow_i_testY =  225
shadow_i_trainX =  525 shadow_i_trainY =  525 shadow_i_testX =  225 shadow_i_testY =  225
shadow_i_trainX =  525 shadow_i_trainY =  525 shadow_i_testX =  225 shadow_i_testY =  225
shadow_i_trainX =  525 shadow_i_trainY =  525 shadow_i_testX =  225 shadow_i_testY =  225
shadow_i_trainX =  525 shadow_i_trainY =  525 shadow_i_testX =  225 shadow_i_testY =  225
shadow_i_trainX =  525 shadow_i_trainY =  525 shadow_i_testX =  225 shadow_i_testY =  225
shadow_i_trainX =  525 shadow_i_trainY =  525 shadow_i_testX =  225 shadow_i_testY =  225
shadow_i_trainX =  525 shadow_i_trainY =  525 shadow_i_testX =  225 shadow_i_testY =  225
shadow_i_t

In [405]:
#train attack model
attack_test_data = prepare_attack_test_data(attack_test_members, attack_test_nonmembers, target_model, is_synthetic)
mem_validation, nmem_validation = prep_validation_data(attack_test_data)
attack_train_df = prep_attack_train_data(n_attack_data)
pred_membership, ori_membership, TP_idx_list, TN_idx_list = shokri_attack(attack_train_df, mem_validation, nmem_validation)
tp, fp, fn, tn, precision, advj, acc, recall = prety_print_result (ori_membership,pred_membership)
print('Accuracy: ', acc, 'Precision: ', precision)

8/8 [==============================] - 0s 2ms/step

 pred (array([[9.93186951e-01, 6.81309216e-03],
       [9.99996603e-01, 3.34540414e-06],
       [5.33304572e-01, 4.66695338e-01],
       [8.33065331e-01, 1.66934624e-01],
       [9.99999821e-01, 6.89283510e-08],
       [9.99379456e-01, 6.20536448e-04],
       [9.98037875e-01, 1.96207990e-03],
       [9.99924123e-01, 7.57550661e-05],
       [9.98999298e-01, 1.00067887e-03],
       [9.99740839e-01, 2.59266293e-04],
       [9.27459240e-01, 7.25407079e-02],
       [9.97939765e-01, 2.06026668e-03],
       [1.42221001e-03, 9.98577833e-01],
       [7.65948653e-01, 2.34051391e-01],
       [9.99999940e-01, 2.36470328e-08],
       [9.83341634e-02, 9.01665688e-01],
       [8.42344575e-03, 9.91576552e-01],
       [9.86712217e-01, 1.32877463e-02],
       [8.72559845e-01, 1.27440065e-01],
       [9.91667211e-01, 8.33283830e-03],
       [9.98672605e-01, 1.32740464e-03],
       [9.99459386e-01, 5.40558773e-04],
       [9.99971330e-01, 2.85528786e-05]

7/7 [==============================] - 0s 1ms/step
1
2/2 [==============================] - 0s 4ms/step
TP: 228     FP: 208     FN: 22     TN: 42
PPV: 0.5229
Advantage: 0.0800
Accuracy:  0.54 Precision:  0.5229357798165137


In [ ]:
# --------------------------------------------Synthetic Data------------------------------------------------------------#

In [406]:
# synthetic dataset
train_size = 500
attack_test_size = 250
target_dataset = pd.read_csv('data/adult_sds.csv', na_values=["?"], header=None)
shadow_dataset = pd.read_csv('data/adultODS10K_to_25K.csv', na_values=["?"], header=None)
attack_test_nonmembers = pd.read_csv('data/adultODS25K_to_30K.csv', na_values=["?"], header=None)
attack_test_nonmembers = attack_test_nonmembers.sample(n=attack_test_size, replace=False)
attack_test_members = pd.read_csv('data/adultODS1_to_5K.csv', na_values=["?"], header=None)


In [47]:
train_size = 7000
target_dataset = pd.read_csv('data/adult_sds_cac.csv', na_values=["?"], header=None)

In [48]:
# trian target model
per_class_sample=5000
channel=0   
EPS=200
act_layer=3
n_class = 2
is_synthetic = True
VERBOSE = 0
test_ratio = 0.3

target_model, dim = train_target_model(target_dataset, per_class_sample, EPS, act_layer, n_class, is_synthetic, train_size)


 Target Train acc :  88.42856884002686 Target Test acc :  85.10000109672546


In [408]:
#train shadow model
n_shadow_models = 20
shadow_data_size = 750

load_shadow_data(shadow_dataset, n_shadow_models, shadow_data_size, test_ratio, is_synthetic)
n_shadow_train_performance, n_shadow_test_performance, n_attack_data, x_shadow_train, y_shadow_train, x_shadow_test, y_shadow_test, shadow_model_init, shadow_accuracy = train_shadow_models(n_shadow_models, n_class, dim, channel)

shadow_i_trainX =  525 shadow_i_trainY =  525 shadow_i_testX =  225 shadow_i_testY =  225
shadow_i_trainX =  525 shadow_i_trainY =  525 shadow_i_testX =  225 shadow_i_testY =  225
shadow_i_trainX =  525 shadow_i_trainY =  525 shadow_i_testX =  225 shadow_i_testY =  225
shadow_i_trainX =  525 shadow_i_trainY =  525 shadow_i_testX =  225 shadow_i_testY =  225
shadow_i_trainX =  525 shadow_i_trainY =  525 shadow_i_testX =  225 shadow_i_testY =  225
shadow_i_trainX =  525 shadow_i_trainY =  525 shadow_i_testX =  225 shadow_i_testY =  225
shadow_i_trainX =  525 shadow_i_trainY =  525 shadow_i_testX =  225 shadow_i_testY =  225
shadow_i_trainX =  525 shadow_i_trainY =  525 shadow_i_testX =  225 shadow_i_testY =  225
shadow_i_trainX =  525 shadow_i_trainY =  525 shadow_i_testX =  225 shadow_i_testY =  225
shadow_i_trainX =  525 shadow_i_trainY =  525 shadow_i_testX =  225 shadow_i_testY =  225
shadow_i_trainX =  525 shadow_i_trainY =  525 shadow_i_testX =  225 shadow_i_testY =  225
shadow_i_t

In [409]:
#train attack model
attack_test_data = prepare_attack_test_data(attack_test_members, attack_test_nonmembers, target_model, is_synthetic)
mem_validation, nmem_validation = prep_validation_data(attack_test_data)
attack_train_df = prep_attack_train_data(n_attack_data)
pred_membership, ori_membership, TP_idx_list, TN_idx_list = shokri_attack(attack_train_df, mem_validation, nmem_validation)
tp, fp, fn, tn, precision, advj, acc, recall = prety_print_result (ori_membership,pred_membership)
print('Accuracy: ', acc, 'Precision: ', precision)

8/8 [==============================] - 0s 2ms/step

 pred (array([[9.99557793e-01, 4.42208780e-04],
       [3.47070545e-01, 6.52929485e-01],
       [9.99999821e-01, 6.52376428e-08],
       [9.99990165e-01, 9.73688657e-06],
       [9.80585933e-01, 1.94141194e-02],
       [9.99998987e-01, 9.07824017e-07],
       [9.99995291e-01, 4.70001305e-06],
       [9.99981701e-01, 1.82892691e-05],
       [9.99839067e-01, 1.60868527e-04],
       [7.04952061e-01, 2.95047909e-01],
       [9.99254346e-01, 7.45724828e-04],
       [9.99211967e-01, 7.88065838e-04],
       [9.96637583e-01, 3.36239929e-03],
       [9.99999940e-01, 3.40392603e-08],
       [9.99998987e-01, 9.28207101e-07],
       [9.95667636e-01, 4.33224207e-03],
       [1.13499640e-02, 9.88650084e-01],
       [1.79487653e-03, 9.98205066e-01],
       [5.18777315e-03, 9.94812191e-01],
       [9.99991238e-01, 8.64630329e-06],
       [1.40724529e-03, 9.98592734e-01],
       [9.99790907e-01, 2.09024132e-04],
       [4.72671054e-02, 9.52732921e-01]

7/7 [==============================] - 0s 1ms/step
1
2/2 [==============================] - 0s 4ms/step
TP: 206     FP: 195     FN: 44     TN: 55
PPV: 0.5137
Advantage: 0.0440
Accuracy:  0.522 Precision:  0.513715710723192


In [ ]:
# --------------------------------------------Synthetic Data-------------------------------------------------------------#

In [ ]:
# --------------------------------------------Overfitting Experiment-----------------------------------------------------#

In [393]:
# original dataset with random index
data = pd.read_csv('data/adult.data', na_values=["?"], header=None)
data.dropna(inplace=True)
target_dataset = data.sample(n = 10000, replace = False)
df_rest = data.loc[~data.index.isin(target_dataset.index)]
shadow_dataset = df_rest.sample(n = 12000, replace = False)
df_rest = df_rest.loc[~df_rest.index.isin(shadow_dataset.index)]
attack_test_nonmembers = df_rest.sample(n = 5000, replace = False)
attack_test_members =  target_dataset.iloc[:7000,:].sample(n = 5000, replace = False)

In [394]:
def load_target_data(dataset, training_data_size, test_ratio, is_synthetic):
    x, y, dim = transform_data(dataset, is_synthetic)
    
    #trainX,testX, trainY, testY = train_test_split(x, y, test_size=test_ratio, random_state=0, stratify=y)
    trainX = x[0:training_data_size,]
    testX = x[5000:,]
    trainY = y[0:training_data_size,]
    testY = y[5000:,]
    return (trainX, trainY), (testX, testY), dim

In [397]:
# trian target model
training_data_size = [2,5,10,15,20,25,50,100,200,500,1000,2000,5000]
per_class_sample=5000
channel=0   
EPS=200
act_layer=2
n_class = 2
is_synthetic = False
VERBOSE = 0
test_ratio = 0.3

accuracy_df = pd.DataFrame()
accuracy_df['training data size'] = np.nan
accuracy_df['training accuracy'] = np.nan
accuracy_df['test accuracy'] = np.nan
accuracy_df['error'] = np.nan

data = pd.read_csv('data/adult.data', na_values=["?"], header=None)
data.dropna(inplace=True)

for j in training_data_size:
    print('\n Training Data Size: ', j, '\n')
    for i in range(10):
        target_dataset = data.sample(n = 10000, replace = False)
        (target_trainX, target_trainY), (target_testX, target_testY), dim = load_target_data(target_dataset, j, test_ratio, is_synthetic)
        target_model,_ = build_simple_mlp(n_class,dim, channel)
        #get_trained_keras_models(model, (target_trainX, target_trainY), (target_testX, target_testY), num_models=1)
        history = target_model.fit(target_trainX, target_trainY, epochs=EPS, batch_size=32, verbose=VERBOSE)
        score = target_model.evaluate(target_testX, target_testY, verbose=VERBOSE)
        _, train_acc = target_model.evaluate(target_trainX, target_trainY, verbose=VERBOSE)
        _, test_acc = target_model.evaluate(target_testX, target_testY, verbose=VERBOSE)
        print('Iteration ', i, "Target Train acc : ", (train_acc * 100.0),"Target Test acc : ", (test_acc * 100.0))
        accuracy_df = accuracy_df.append({'training data size':j, 'training accuracy' : (train_acc * 100.0), 'test accuracy': (test_acc * 100.0), 'error': ((train_acc * 100.0)-(test_acc * 100.0))}, ignore_index=True)
        #print('\n', 'Model test accuracy:', score[1])
        



 Training Data Size:  2 

Iteration  0 Target Train acc :  100.0 Target Test acc :  63.599997758865356
Iteration  1 Target Train acc :  100.0 Target Test acc :  37.43999898433685
Iteration  2 Target Train acc :  100.0 Target Test acc :  75.95999836921692
Iteration  3 Target Train acc :  100.0 Target Test acc :  24.220000207424164
Iteration  4 Target Train acc :  100.0 Target Test acc :  55.86000084877014
Iteration  5 Target Train acc :  100.0 Target Test acc :  77.48000025749207
Iteration  6 Target Train acc :  100.0 Target Test acc :  40.939998626708984
Iteration  7 Target Train acc :  100.0 Target Test acc :  42.739999294281006
Iteration  8 Target Train acc :  100.0 Target Test acc :  41.31999909877777
Iteration  9 Target Train acc :  100.0 Target Test acc :  76.03999972343445

 Training Data Size:  5 

Iteration  0 Target Train acc :  100.0 Target Test acc :  29.600000381469727
Iteration  1 Target Train acc :  100.0 Target Test acc :  62.81999945640564
Iteration  2 Target Train acc

Iteration  0 Target Train acc :  93.59999895095825 Target Test acc :  81.73999786376953
Iteration  1 Target Train acc :  93.09999942779541 Target Test acc :  82.09999799728394
Iteration  2 Target Train acc :  91.60000085830688 Target Test acc :  80.64000010490417
Iteration  3 Target Train acc :  90.89999794960022 Target Test acc :  81.90000057220459
Iteration  4 Target Train acc :  92.10000038146973 Target Test acc :  82.95999765396118
Iteration  5 Target Train acc :  93.4000015258789 Target Test acc :  80.50000071525574
Iteration  6 Target Train acc :  93.19999814033508 Target Test acc :  82.34000205993652
Iteration  7 Target Train acc :  94.70000267028809 Target Test acc :  81.13999962806702
Iteration  8 Target Train acc :  94.70000267028809 Target Test acc :  81.98000192642212
Iteration  9 Target Train acc :  96.10000252723694 Target Test acc :  80.68000078201294

 Training Data Size:  2000 

Iteration  0 Target Train acc :  91.20000004768372 Target Test acc :  82.03999996185303
Ite

In [398]:
#accuracy_df =pd.read_csv('data/adult_overfitting_test_trainsize')
training_data_size = [2,5,10,15,20,25,50,100,200,500,1000,2000,5000]

avg_error_ci95_df = pd.DataFrame()
avg_error_ci95_df['training data size'] = np.nan
avg_error_ci95_df['average training accuracy'] = np.nan
avg_error_ci95_df['average test accuracy'] = np.nan
avg_error_ci95_df['average error'] = np.nan
avg_error_ci95_df['ci95 low'] = np.nan
avg_error_ci95_df['ci95 high'] = np.nan

for i in training_data_size:
    error = np.array(accuracy_df.loc[accuracy_df['training data size'] == i, 'error'])
    training_accuracy = np.array(accuracy_df.loc[accuracy_df['training data size'] == i, 'training accuracy'])
    test_accuracy = np.array(accuracy_df.loc[accuracy_df['training data size'] == i, 'test accuracy'])
    ci95 = st.t.interval(alpha=0.95, df=len(error)-1, loc=np.mean(error), scale=st.sem(error))
    row = pd.DataFrame({'training data size': [i], 'average training accuracy': \
            [np.mean(training_accuracy)], 'average test accuracy': [np.mean(test_accuracy)], 'average error': [np.mean(error)],\
                                                  'ci95 low': [ci95[0]], 'ci95 high': [ci95[1]]})
    avg_error_ci95_df = pd.concat([avg_error_ci95_df, row], axis=0, ignore_index=True)

In [399]:
avg_error_ci95_df

,training data size,average training accuracy,average test accuracy,average error,ci95 low,ci95 high
0,2.0,100.000000,53.559999,46.440001,32.883379,59.996623
1,5.0,100.000000,64.442000,35.558000,25.182118,45.933883
2,10.0,100.000000,69.211999,30.788001,26.872650,34.703351
3,15.0,100.000000,71.778001,28.221999,24.323098,32.120901
4,20.0,100.000000,70.580000,29.420000,24.609671,34.230330
5,25.0,100.000000,74.212000,25.788000,22.316257,29.259742
6,50.0,99.400000,75.656001,23.743999,21.182247,26.305751
7,100.0,96.800001,78.802000,17.998001,15.158599,20.837403
8,200.0,96.400000,79.517999,16.882001,14.401084,19.362918
9,500.0,95.759999,79.634000,16.125999,14.616011,17.635988


In [ ]:
# --------------------------------------------Overfitting Experiment-----------------------------------------------------#

In [ ]:
# -------------------------------------------- DNN Experiment --------------------------------------------------------#

In [76]:
def train_target_model(target_dataset, per_class_sample, epoch, act_layer, n_class, is_synthetic, train_size, channel=0, verbose=0, test_ratio=0.3):
    
    (target_trainX, target_trainY), (target_testX, target_testY), dim = load_target_data(target_dataset, train_size, test_ratio, is_synthetic)
    target_model,_ = build_dnn(n_class,dim)
    #get_trained_keras_models(model, (target_trainX, target_trainY), (target_testX, target_testY), num_models=1)
    history = target_model.fit(target_trainX, target_trainY, epochs=EPS, batch_size=32, verbose=VERBOSE)
    score = target_model.evaluate(target_testX, target_testY, verbose=VERBOSE)
    _, train_acc = target_model.evaluate(target_trainX, target_trainY, verbose=VERBOSE)
    _, test_acc = target_model.evaluate(target_testX, target_testY, verbose=VERBOSE)
    print('\n', "Target Train acc : ", (train_acc * 100.0),"Target Test acc : ", (test_acc * 100.0))
    #print('\n', 'Model test accuracy:', score[1])
    return target_model, dim

In [77]:
def train_shadow_models(n_shadow, n_class, dim, channel):
    full_sm_train_pred=[]
    full_sm_train_class=[]
    
    full_sm_test_pred=[]
    full_sm_test_class=[]
    
    full_clz_train=[]
    full_clz_test=[]
    
    members=[]
    nonmembers=[]
    
    train_accuracy=[]
    test_accuracy=[]

    for j in range(n_shadow):
        
        print("Shadow Model ", j)
        
        print('Training shadow model {}'.format(j))
        data = read_data('shadow_adult{}_data.npz'.format(j))
        x_shadow_train, y_shadow_train, x_shadow_test, y_shadow_test = data
        #print('x_shadow trian\n', x_shadow_train,'\n y_shadow trian\n', y_shadow_train, '\n x_shadow test\n', x_shadow_test, '\n y_shadow test\n', y_shadow_test)

        model, act_layer = build_dnn(n_class,dim)
            
            
        # fit model
        history = model.fit(x_shadow_train, y_shadow_train, epochs=EPS, batch_size=32, validation_data=(x_shadow_test, y_shadow_test), verbose=0)
    
        # evaluate model
        _, train_acc = model.evaluate(x_shadow_train, y_shadow_train, verbose=0)
        _, test_acc = model.evaluate(x_shadow_test, y_shadow_test, verbose=0)
        print("Shadow Train acc : ", (train_acc * 100.0),"Shadow Test acc : ", (test_acc * 100.0))
        train_accuracy.append((train_acc * 100.0))
        test_accuracy.append((test_acc * 100.0))

    
        #train SM
        sm_train_pred=model.predict(x_shadow_train, batch_size=32)
        sm_train_class=np.argmax(y_shadow_train,axis=1)
    
    
        #test SM
        sm_test_pred=model.predict(x_shadow_test, batch_size=32)
        sm_test_class=np.argmax(y_shadow_test,axis=1)
        
     
        full_sm_train_pred.append(sm_train_pred)        
        full_sm_train_class.append(sm_train_class)
        members.append(np.ones(len(sm_train_pred)))
        
        full_sm_test_pred.append(sm_test_pred)        
        full_sm_test_class.append(sm_test_class) 
        nonmembers.append(np.zeros(len(sm_test_pred)))


    full_sm_train_pred = np.vstack(full_sm_train_pred)
    full_sm_train_class = [item for sublist in full_sm_train_class for item in sublist]
    members = [item for sublist in members for item in sublist]
    
    full_sm_test_pred = np.vstack(full_sm_test_pred)
    full_sm_test_class = [item for sublist in full_sm_test_class for item in sublist]
    nonmembers = [item for sublist in nonmembers for item in sublist]
    
    shadow_train_performance=(full_sm_train_pred, np.array(full_sm_train_class))
    shadow_test_performance=(full_sm_test_pred, np.array(full_sm_test_class))


    ###atack data preparation
    attack_x = (full_sm_train_pred,full_sm_test_pred)
    #attack_x = np.vstack(attack_x)
    
    attack_y = (np.array(members).astype('int32'),np.array(nonmembers).astype('int32'))
    #attack_y = np.concatenate(attack_y)
    #attack_y = attack_y.astype('int32')
    
    
    classes = (np.array(full_sm_train_class),np.array(full_sm_test_class))
    #classes = np.array([item for sublist in classes for item in sublist])


    attack_dataset = (attack_x,attack_y,classes)
    shadow_accuracy = (train_accuracy, test_accuracy)

            
    return  shadow_train_performance, shadow_test_performance, attack_dataset, x_shadow_train, y_shadow_train, x_shadow_test, y_shadow_test, model, shadow_accuracy

In [78]:
train_size = 7000
attack_test_size = 5000
data = pd.read_csv('data/adult.data', na_values=["?"], header=None)
data.dropna(inplace=True)
target_dataset = data.sample(n = 10000, replace = False)
df_rest = data.loc[~data.index.isin(target_dataset.index)]
shadow_dataset = df_rest.sample(n = 12000, replace = False)
df_rest = df_rest.loc[~df_rest.index.isin(shadow_dataset.index)]
attack_test_nonmembers = df_rest.sample(n = attack_test_size, replace = False)
attack_test_members =  target_dataset.iloc[:train_size,:].sample(n = attack_test_size, replace = False)

In [79]:
# trian target model
per_class_sample=5000
channel=0   
EPS=200
act_layer=6
n_class = 2
is_synthetic = False
VERBOSE = 0
test_ratio = 0.3

target_model, dim = train_target_model(target_dataset, per_class_sample, EPS, act_layer, n_class, is_synthetic, train_size)


 Target Train acc :  85.62856912612915 Target Test acc :  83.63333344459534


In [80]:
#train shadow model
n_shadow_models = 20
shadow_data_size = 10000

load_shadow_data(shadow_dataset, n_shadow_models, shadow_data_size, test_ratio, is_synthetic)
n_shadow_train_performance, n_shadow_test_performance, n_attack_data, x_shadow_train, y_shadow_train, x_shadow_test, y_shadow_test, shadow_model_init, shadow_accuracy = train_shadow_models(n_shadow_models, n_class, dim, channel)

shadow_i_trainX =  7000 shadow_i_trainY =  7000 shadow_i_testX =  3000 shadow_i_testY =  3000
shadow_i_trainX =  7000 shadow_i_trainY =  7000 shadow_i_testX =  3000 shadow_i_testY =  3000
shadow_i_trainX =  7000 shadow_i_trainY =  7000 shadow_i_testX =  3000 shadow_i_testY =  3000
shadow_i_trainX =  7000 shadow_i_trainY =  7000 shadow_i_testX =  3000 shadow_i_testY =  3000
shadow_i_trainX =  7000 shadow_i_trainY =  7000 shadow_i_testX =  3000 shadow_i_testY =  3000
shadow_i_trainX =  7000 shadow_i_trainY =  7000 shadow_i_testX =  3000 shadow_i_testY =  3000
shadow_i_trainX =  7000 shadow_i_trainY =  7000 shadow_i_testX =  3000 shadow_i_testY =  3000
shadow_i_trainX =  7000 shadow_i_trainY =  7000 shadow_i_testX =  3000 shadow_i_testY =  3000
shadow_i_trainX =  7000 shadow_i_trainY =  7000 shadow_i_testX =  3000 shadow_i_testY =  3000
shadow_i_trainX =  7000 shadow_i_trainY =  7000 shadow_i_testX =  3000 shadow_i_testY =  3000
shadow_i_trainX =  7000 shadow_i_trainY =  7000 shadow_i_tes

In [81]:
#train attack model
attack_test_data = prepare_attack_test_data(attack_test_members, attack_test_nonmembers, target_model, is_synthetic)
mem_validation, nmem_validation = prep_validation_data(attack_test_data)
attack_train_df = prep_attack_train_data(n_attack_data)
pred_membership, ori_membership, TP_idx_list, TN_idx_list = shokri_attack(attack_train_df, mem_validation, nmem_validation)
tp, fp, fn, tn, precision, advj, acc, recall = prety_print_result (ori_membership,pred_membership)
print('Accuracy: ', acc, 'Precision: ', precision)

157/157 [==============================] - 1s 4ms/step

 pred (array([[0.994057  , 0.00594292],
       [0.08962913, 0.9103709 ],
       [0.96087474, 0.03912532],
       ...,
       [0.86039567, 0.13960423],
       [0.9942621 , 0.00573787],
       [0.99465257, 0.00534731]], dtype=float32), array([[0.99454147, 0.00545851],
       [0.69537467, 0.30462536],
       [0.728546  , 0.27145386],
       ...,
       [0.84629935, 0.15370063],
       [0.7286267 , 0.27137324],
       [0.97372544, 0.02627459]], dtype=float32))

 class (array([0, 1, 0, ..., 0, 0, 0]), array([0, 0, 0, ..., 0, 0, 0]))

 mem status (array([1, 1, 1, ..., 1, 1, 1], dtype=int32), array([0, 0, 0, ..., 0, 0, 0], dtype=int32))
0
120/120 [==============================] - 0s 2ms/step
1
38/38 [==============================] - 0s 1ms/step
TP: 5000     FP: 4999     FN: 0     TN: 1
PPV: 0.5001
Advantage: 0.0002
Accuracy:  0.5001 Precision:  0.5000500050005
